# Example training and inference

In [ ]:
import json, codecs
import random

from TransformerLanguageModel import CustomTokenizer, TransformerLanguageModel, Trainer
from nltk.tokenize import word_tokenize

In [ ]:
start_token = '<S>'
end_token = '</S>'
pad_token = '<PAD>'
vocab_size = 30000
device = 'cuda'

## Loading data

In [ ]:
with open('data/dataset.json', 'r', encoding="utf-8") as f:
    examples = json.load(f)

split = int(0.05*len(examples))
dev = examples[:split]
train = examples[split:]

## Training tokenizer

In [ ]:
tokenizer = CustomTokenizer(pad_token, start_token, end_token, vocab_size=vocab_size)
tokenizer.train(train)
tokenizer.save('tokenizer.json')

When the tokenizer is already trained. Load it with:

In [ ]:
tokenizer.load('tokenizer.json')

## Training model

In [ ]:
lm = TransformerLanguageModel('lm', n_head=6, d_model=256, d_ff=4*256, layer_count=5, embed_dropout=0.3, cell_dropout=0.2, tokenizer=tokenizer).to(device)

trainer = Trainer(vocab_size, 16, 10)
trainer.train(lm, train, dev)

Extra training of the model

In [ ]:
lm.load('lm')
trainer = Trainer(vocab_size, 16, 10)
trainer.train(lm, train, dev, extra_train=True)

## Inference
With nucleus sampling with inference parameter 0.9

In [ ]:
print(lm.generate_sentence(word_tokenize('Hello'), True, 'nuc', inference_parameter=0.9, limit=100))